# **Загрузка данных**

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
%cd /content/drive/My Drive/Colab Notebooks/Files/

/content/drive/My Drive/Colab Notebooks/Files


In [ ]:
lem_tfidf = pd.read_csv('hyperc_lem_tfidf.csv')
valence = pd.read_csv('hyperc_lem_valence.csv')
mfcc = pd.read_csv('hyperc_mfcc.csv')

In [ ]:
valence['valence'] = valence['valence'].replace({'neut': 1, 'pos': 2, 'neg': 0})
valence_list = valence['valence'].tolist()

In [ ]:
lem_tfidf = lem_tfidf.to_numpy()
mfcc = mfcc.to_numpy()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.decomposition import PCA
pca = PCA()
pca.fit(mfcc)

PCA()

In [ ]:
cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
n_components = np.argmax(cumulative_variance >= 0.9) + 1

In [ ]:
pca = PCA(n_components=n_components)
pca_mfcc = pca.fit_transform(mfcc)

In [ ]:
X_text = lem_tfidf
X_text_train, X_text_test, y_train, y_test = train_test_split(X_text, valence_list, test_size=0.3, random_state=28)

X_audio = pca_mfcc
X_audio_train, X_audio_test, y_train, y_test = train_test_split(X_audio, valence_list, test_size=0.3, random_state=28)

In [ ]:
#импорт библиотек
import torch
import torchvision
from torchvision import transforms, datasets
import torch.nn as nn
import torch.nn.functional as F
import scipy
import torch.utils.data as data_utils
from sklearn.metrics import classification_report
import torch.optim as optim
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

# **Текстовая классификация**

In [ ]:
#преобразование в тензоры
inputs_train = torch.tensor(X_text_train)
targets_train = torch.IntTensor(y_train)

inputs_test = torch.tensor(X_text_test)
targets_test = torch.IntTensor(y_test)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True)
testset = torch.utils.data.DataLoader(test, shuffle=False)

**3 слоя: 128, 64, 16**

ReLu

Adam

In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(1000, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 16)
        self.fc4 = nn.Linear(16, 3)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)

In [ ]:
model = MLP()

# Определение функции потерь и оптимизатора
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Обучение модели
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in trainset:
        optimizer.zero_grad()
        outputs = model(inputs.float())
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(trainset)}")

Epoch 1/10, Loss: 0.4475890406476107
Epoch 2/10, Loss: 0.27776610399802815
Epoch 3/10, Loss: 0.19737205203803995
Epoch 4/10, Loss: 0.1487340255565448
Epoch 5/10, Loss: 0.1230827178051925
Epoch 6/10, Loss: 0.10371145612151621
Epoch 7/10, Loss: 0.0851347414685236
Epoch 8/10, Loss: 0.08082380181043983
Epoch 9/10, Loss: 0.06835724387256237
Epoch 10/10, Loss: 0.06661412860360987


In [ ]:
model.eval()  # Устанавливаем модель в режим оценки (выключаем режим обучения)

all_predictions = []
all_targets = []

with torch.no_grad():
    for inputs, labels in testset:
        outputs = model(inputs.float())
        _, predicted = torch.max(outputs, 1)
        all_predictions.extend(predicted.cpu().numpy())
        all_targets.extend(labels.cpu().numpy())

acc_exp1_1 = accuracy_score(all_targets, all_predictions)
f1_exp1_1 = f1_score(all_targets, all_predictions, average='weighted')
print(classification_report(all_targets, all_predictions))

              precision    recall  f1-score   support

           0       1.00      0.12      0.22         8
           1       0.91      0.95      0.93       222
           2       0.31      0.24      0.27        21

    accuracy                           0.87       251
   macro avg       0.74      0.44      0.47       251
weighted avg       0.86      0.87      0.85       251



**3 слоя: 128, 64, 16**

Tanh

Adam

In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(1000, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 16)
        self.fc4 = nn.Linear(16, 3)

    def forward(self, x):
        x = F.tanh(self.fc1(x))
        x = F.tanh(self.fc2(x))
        x = F.tanh(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)

In [ ]:
model = MLP()

# Определение функции потерь и оптимизатора
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Обучение модели
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in trainset:
        optimizer.zero_grad()
        outputs = model(inputs.float())
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(trainset)}")

Epoch 1/10, Loss: 0.3799664510244673
Epoch 2/10, Loss: 0.2880519080498155
Epoch 3/10, Loss: 0.18098056452160327
Epoch 4/10, Loss: 0.13961694191096552
Epoch 5/10, Loss: 0.1153475184059074
Epoch 6/10, Loss: 0.09060964902054588
Epoch 7/10, Loss: 0.074212801466795
Epoch 8/10, Loss: 0.06576942201259692
Epoch 9/10, Loss: 0.06964387507980616
Epoch 10/10, Loss: 0.06381694835522428


In [ ]:
model.eval()  # Устанавливаем модель в режим оценки (выключаем режим обучения)

all_predictions = []
all_targets = []

with torch.no_grad():
    for inputs, labels in testset:
        outputs = model(inputs.float())
        _, predicted = torch.max(outputs, 1)
        all_predictions.extend(predicted.cpu().numpy())
        all_targets.extend(labels.cpu().numpy())

acc_exp1_2 = accuracy_score(all_targets, all_predictions)
f1_exp1_2 = f1_score(all_targets, all_predictions, average='weighted')
print(classification_report(all_targets, all_predictions))

              precision    recall  f1-score   support

           0       0.50      0.25      0.33         8
           1       0.91      0.95      0.93       222
           2       0.33      0.24      0.28        21

    accuracy                           0.87       251
   macro avg       0.58      0.48      0.51       251
weighted avg       0.85      0.87      0.86       251



**3 слоя: 128, 64, 16**

Sigmoid

Adam

In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(1000, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 16)
        self.fc4 = nn.Linear(16, 3)

    def forward(self, x):
        x = F.sigmoid(self.fc1(x))
        x = F.sigmoid(self.fc2(x))
        x = F.sigmoid(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)

In [ ]:
model = MLP()

# Определение функции потерь и оптимизатора
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Обучение модели
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in trainset:
        optimizer.zero_grad()
        outputs = model(inputs.float())
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(trainset)}")

Epoch 1/10, Loss: 0.4304330087367503
Epoch 2/10, Loss: 0.35803938378447325
Epoch 3/10, Loss: 0.35803892920959873
Epoch 4/10, Loss: 0.3579750238289964
Epoch 5/10, Loss: 0.3575050120927253
Epoch 6/10, Loss: 0.357929400626832
Epoch 7/10, Loss: 0.35855225183542555
Epoch 8/10, Loss: 0.3580675573051487
Epoch 9/10, Loss: 0.35818051522530575
Epoch 10/10, Loss: 0.35498565012400596


In [ ]:
model.eval()  # Устанавливаем модель в режим оценки (выключаем режим обучения)

all_predictions = []
all_targets = []

with torch.no_grad():
    for inputs, labels in testset:
        outputs = model(inputs.float())
        _, predicted = torch.max(outputs, 1)
        all_predictions.extend(predicted.cpu().numpy())
        all_targets.extend(labels.cpu().numpy())

acc_exp1_3 = accuracy_score(all_targets, all_predictions)
f1_exp1_3 = f1_score(all_targets, all_predictions, average='weighted')
print(classification_report(all_targets, all_predictions))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       0.88      1.00      0.94       222
           2       0.00      0.00      0.00        21

    accuracy                           0.88       251
   macro avg       0.29      0.33      0.31       251
weighted avg       0.78      0.88      0.83       251



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**3 слоя: 128, 64, 16**

ReLu

SGD

In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(1000, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 16)
        self.fc4 = nn.Linear(16, 3)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)

In [ ]:
model = MLP()

# Определение функции потерь и оптимизатора
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [ ]:
# Обучение модели
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in trainset:
        optimizer.zero_grad()
        outputs = model(inputs.float())
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(trainset)}")

Epoch 1/10, Loss: 0.8595611369221329
Epoch 2/10, Loss: 0.6641885637214622
Epoch 3/10, Loss: 0.5431350835089414
Epoch 4/10, Loss: 0.4684435221179681
Epoch 5/10, Loss: 0.4232458541006821
Epoch 6/10, Loss: 0.39673176865328236
Epoch 7/10, Loss: 0.38122200388237704
Epoch 8/10, Loss: 0.37217771986540965
Epoch 9/10, Loss: 0.3669374791160865
Epoch 10/10, Loss: 0.36362162366586115


In [ ]:
model.eval()  # Устанавливаем модель в режим оценки (выключаем режим обучения)

all_predictions = []
all_targets = []

with torch.no_grad():
    for inputs, labels in testset:
        outputs = model(inputs.float())
        _, predicted = torch.max(outputs, 1)
        all_predictions.extend(predicted.cpu().numpy())
        all_targets.extend(labels.cpu().numpy())

acc_exp1_4 = accuracy_score(all_targets, all_predictions)
f1_exp1_4 = f1_score(all_targets, all_predictions, average='weighted')
print(classification_report(all_targets, all_predictions))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       0.88      1.00      0.94       222
           2       0.00      0.00      0.00        21

    accuracy                           0.88       251
   macro avg       0.29      0.33      0.31       251
weighted avg       0.78      0.88      0.83       251



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**3 слоя: 128, 64, 16**

Tanh

SGD

In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(1000, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 16)
        self.fc4 = nn.Linear(16, 3)

    def forward(self, x):
        x = F.tanh(self.fc1(x))
        x = F.tanh(self.fc2(x))
        x = F.tanh(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)

In [ ]:
model = MLP()

# Определение функции потерь и оптимизатора
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [ ]:
# Обучение модели
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in trainset:
        optimizer.zero_grad()
        outputs = model(inputs.float())
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(trainset)}")

Epoch 1/10, Loss: 0.9453481682168885
Epoch 2/10, Loss: 0.6652217655096005
Epoch 3/10, Loss: 0.5084642959602088
Epoch 4/10, Loss: 0.42893321298407855
Epoch 5/10, Loss: 0.39106878280435176
Epoch 6/10, Loss: 0.37322826609398857
Epoch 7/10, Loss: 0.36482032095792977
Epoch 8/10, Loss: 0.3608554410740196
Epoch 9/10, Loss: 0.35890707620267803
Epoch 10/10, Loss: 0.3578016444861787


In [ ]:
model.eval()  # Устанавливаем модель в режим оценки (выключаем режим обучения)

all_predictions = []
all_targets = []

with torch.no_grad():
    for inputs, labels in testset:
        outputs = model(inputs.float())
        _, predicted = torch.max(outputs, 1)
        all_predictions.extend(predicted.cpu().numpy())
        all_targets.extend(labels.cpu().numpy())

acc_exp1_5 = accuracy_score(all_targets, all_predictions)
f1_exp1_5 = f1_score(all_targets, all_predictions, average='weighted')
print(classification_report(all_targets, all_predictions))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       0.88      1.00      0.94       222
           2       0.00      0.00      0.00        21

    accuracy                           0.88       251
   macro avg       0.29      0.33      0.31       251
weighted avg       0.78      0.88      0.83       251



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**3 слоя: 128, 64, 16**

Sigmoid

SGD

In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(1000, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 16)
        self.fc4 = nn.Linear(16, 3)

    def forward(self, x):
        x = F.sigmoid(self.fc1(x))
        x = F.sigmoid(self.fc2(x))
        x = F.sigmoid(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)

In [ ]:
model = MLP()

# Определение функции потерь и оптимизатора
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [ ]:
# Обучение модели
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in trainset:
        optimizer.zero_grad()
        outputs = model(inputs.float())
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(trainset)}")

Epoch 1/10, Loss: 0.6637302746494679
Epoch 2/10, Loss: 0.41272771849231493
Epoch 3/10, Loss: 0.37404867265482095
Epoch 4/10, Loss: 0.36361723232177273
Epoch 5/10, Loss: 0.3600449308650776
Epoch 6/10, Loss: 0.35858687015943624
Epoch 7/10, Loss: 0.35784720490154637
Epoch 8/10, Loss: 0.35734824944881793
Epoch 9/10, Loss: 0.35711196348658963
Epoch 10/10, Loss: 0.35689363281017306


In [ ]:
model.eval()  # Устанавливаем модель в режим оценки (выключаем режим обучения)

all_predictions = []
all_targets = []

with torch.no_grad():
    for inputs, labels in testset:
        outputs = model(inputs.float())
        _, predicted = torch.max(outputs, 1)
        all_predictions.extend(predicted.cpu().numpy())
        all_targets.extend(labels.cpu().numpy())

acc_exp1_6 = accuracy_score(all_targets, all_predictions)
f1_exp1_6 = f1_score(all_targets, all_predictions, average='weighted')
print(classification_report(all_targets, all_predictions))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       0.88      1.00      0.94       222
           2       0.00      0.00      0.00        21

    accuracy                           0.88       251
   macro avg       0.29      0.33      0.31       251
weighted avg       0.78      0.88      0.83       251



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**3 слоя: 128, 64, 16**

ReLu

RMSprop

In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(1000, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 16)
        self.fc4 = nn.Linear(16, 3)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)

In [ ]:
model = MLP()

# Определение функции потерь и оптимизатора
criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(model.parameters(), lr=0.001)

In [ ]:
# Обучение модели
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in trainset:
        optimizer.zero_grad()
        outputs = model(inputs.float())
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(trainset)}")

Epoch 1/10, Loss: 0.3851343958740846
Epoch 2/10, Loss: 0.2967447145591613
Epoch 3/10, Loss: 0.2450964655622107
Epoch 4/10, Loss: 0.21681772003608896
Epoch 5/10, Loss: 0.19145620142588615
Epoch 6/10, Loss: 0.1734741098389447
Epoch 7/10, Loss: 0.15302126853179587
Epoch 8/10, Loss: 0.13292971110460558
Epoch 9/10, Loss: 0.1236175020994178
Epoch 10/10, Loss: 0.11619398685483012


In [ ]:
model.eval()  # Устанавливаем модель в режим оценки (выключаем режим обучения)

all_predictions = []
all_targets = []

with torch.no_grad():
    for inputs, labels in testset:
        outputs = model(inputs.float())
        _, predicted = torch.max(outputs, 1)
        all_predictions.extend(predicted.cpu().numpy())
        all_targets.extend(labels.cpu().numpy())

acc_exp1_7 = accuracy_score(all_targets, all_predictions)
f1_exp1_7 = f1_score(all_targets, all_predictions, average='weighted')
print(classification_report(all_targets, all_predictions))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       0.90      0.97      0.93       222
           2       0.31      0.19      0.24        21

    accuracy                           0.87       251
   macro avg       0.40      0.39      0.39       251
weighted avg       0.82      0.87      0.85       251



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**3 слоя: 128, 64, 16**

Tanh

RMSprop

In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(1000, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 16)
        self.fc4 = nn.Linear(16, 3)

    def forward(self, x):
        x = F.tanh(self.fc1(x))
        x = F.tanh(self.fc2(x))
        x = F.tanh(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)

In [ ]:
model = MLP()

# Определение функции потерь и оптимизатора
criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(model.parameters(), lr=0.001)

In [ ]:
# Обучение модели
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in trainset:
        optimizer.zero_grad()
        outputs = model(inputs.float())
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(trainset)}")

Epoch 1/10, Loss: 0.370210393867096
Epoch 2/10, Loss: 0.2899645166040285
Epoch 3/10, Loss: 0.2319828783054908
Epoch 4/10, Loss: 0.19793769676960352
Epoch 5/10, Loss: 0.16807543376690706
Epoch 6/10, Loss: 0.1350693918223849
Epoch 7/10, Loss: 0.11738911430083061
Epoch 8/10, Loss: 0.09749145876883017
Epoch 9/10, Loss: 0.08150630629674163
Epoch 10/10, Loss: 0.07501259140648733


In [ ]:
model.eval()  # Устанавливаем модель в режим оценки (выключаем режим обучения)

all_predictions = []
all_targets = []

with torch.no_grad():
    for inputs, labels in testset:
        outputs = model(inputs.float())
        _, predicted = torch.max(outputs, 1)
        all_predictions.extend(predicted.cpu().numpy())
        all_targets.extend(labels.cpu().numpy())

acc_exp1_8 = accuracy_score(all_targets, all_predictions)
f1_exp1_8 = f1_score(all_targets, all_predictions, average='weighted')
print(classification_report(all_targets, all_predictions))

              precision    recall  f1-score   support

           0       0.67      0.25      0.36         8
           1       0.91      0.95      0.93       222
           2       0.33      0.24      0.28        21

    accuracy                           0.87       251
   macro avg       0.64      0.48      0.52       251
weighted avg       0.85      0.87      0.86       251



**3 слоя: 128, 64, 16**

Sigmoid

RMSprop

In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(1000, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 16)
        self.fc4 = nn.Linear(16, 3)

    def forward(self, x):
        x = F.sigmoid(self.fc1(x))
        x = F.sigmoid(self.fc2(x))
        x = F.sigmoid(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)

In [ ]:
model = MLP()

# Определение функции потерь и оптимизатора
criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(model.parameters(), lr=0.001)

In [ ]:
# Обучение модели
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in trainset:
        optimizer.zero_grad()
        outputs = model(inputs.float())
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(trainset)}")

Epoch 1/10, Loss: 0.40601279937996054
Epoch 2/10, Loss: 0.35876159566753313
Epoch 3/10, Loss: 0.3570342402438766
Epoch 4/10, Loss: 0.35943105851346385
Epoch 5/10, Loss: 0.35844150046286
Epoch 6/10, Loss: 0.35859310021224605
Epoch 7/10, Loss: 0.35583272680486866
Epoch 8/10, Loss: 0.35771780126883074
Epoch 9/10, Loss: 0.34919604016899247
Epoch 10/10, Loss: 0.3274179904771199


In [ ]:
model.eval()  # Устанавливаем модель в режим оценки (выключаем режим обучения)

all_predictions = []
all_targets = []

with torch.no_grad():
    for inputs, labels in testset:
        outputs = model(inputs.float())
        _, predicted = torch.max(outputs, 1)
        all_predictions.extend(predicted.cpu().numpy())
        all_targets.extend(labels.cpu().numpy())

acc_exp1_9 = accuracy_score(all_targets, all_predictions)
f1_exp1_9 = f1_score(all_targets, all_predictions, average='weighted')
print(classification_report(all_targets, all_predictions))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       0.88      1.00      0.94       222
           2       0.00      0.00      0.00        21

    accuracy                           0.88       251
   macro avg       0.29      0.33      0.31       251
weighted avg       0.78      0.88      0.83       251



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# **Аудио классификация**

In [ ]:
#преобразование в тензоры
inputs_train = torch.tensor(X_audio_train)
targets_train = torch.IntTensor(y_train)

inputs_test = torch.tensor(X_audio_test)
targets_test = torch.IntTensor(y_test)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True)
testset = torch.utils.data.DataLoader(test, shuffle=False)

**3 слоя: 128, 64, 16**

ReLu

Adam

In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(32, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 16)
        self.fc4 = nn.Linear(16, 3)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)

In [ ]:
model = MLP()

# Определение функции потерь и оптимизатора
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Обучение модели
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in trainset:
        optimizer.zero_grad()
        outputs = model(inputs.float())
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(trainset)}")

Epoch 1/10, Loss: 0.4463722922435383
Epoch 2/10, Loss: 0.38482827778885564
Epoch 3/10, Loss: 0.3544988789380514
Epoch 4/10, Loss: 0.3437883674578624
Epoch 5/10, Loss: 0.34211601313947493
Epoch 6/10, Loss: 0.33261289625098805
Epoch 7/10, Loss: 0.32145842749166653
Epoch 8/10, Loss: 0.3264840546663318
Epoch 9/10, Loss: 0.31450807289217464
Epoch 10/10, Loss: 0.33324675558191474


In [ ]:
model.eval()  # Устанавливаем модель в режим оценки (выключаем режим обучения)

all_predictions = []
all_targets = []

with torch.no_grad():
    for inputs, labels in testset:
        outputs = model(inputs.float())
        _, predicted = torch.max(outputs, 1)
        all_predictions.extend(predicted.cpu().numpy())
        all_targets.extend(labels.cpu().numpy())

acc_exp1_1 = accuracy_score(all_targets, all_predictions)
f1_exp1_1 = f1_score(all_targets, all_predictions, average='weighted')
print(classification_report(all_targets, all_predictions))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       0.88      1.00      0.94       222
           2       0.00      0.00      0.00        21

    accuracy                           0.88       251
   macro avg       0.29      0.33      0.31       251
weighted avg       0.78      0.88      0.83       251



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**3 слоя: 128, 64, 16**

Tanh

Adam

In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(32, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 16)
        self.fc4 = nn.Linear(16, 3)

    def forward(self, x):
        x = F.tanh(self.fc1(x))
        x = F.tanh(self.fc2(x))
        x = F.tanh(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)

In [ ]:
model = MLP()

# Определение функции потерь и оптимизатора
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Обучение модели
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in trainset:
        optimizer.zero_grad()
        outputs = model(inputs.float())
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(trainset)}")

Epoch 1/10, Loss: 0.5174870617035936
Epoch 2/10, Loss: 0.36671889520938766
Epoch 3/10, Loss: 0.35485175159307736
Epoch 4/10, Loss: 0.35448136840712147
Epoch 5/10, Loss: 0.34527118563473125
Epoch 6/10, Loss: 0.344507244870859
Epoch 7/10, Loss: 0.33705004102727765
Epoch 8/10, Loss: 0.34184493010106465
Epoch 9/10, Loss: 0.33960892724617675
Epoch 10/10, Loss: 0.3294755499266842


In [ ]:
model.eval()  # Устанавливаем модель в режим оценки (выключаем режим обучения)

all_predictions = []
all_targets = []

with torch.no_grad():
    for inputs, labels in testset:
        outputs = model(inputs.float())
        _, predicted = torch.max(outputs, 1)
        all_predictions.extend(predicted.cpu().numpy())
        all_targets.extend(labels.cpu().numpy())

acc_exp1_2 = accuracy_score(all_targets, all_predictions)
f1_exp1_2 = f1_score(all_targets, all_predictions, average='weighted')
print(classification_report(all_targets, all_predictions))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       0.88      1.00      0.94       222
           2       0.00      0.00      0.00        21

    accuracy                           0.88       251
   macro avg       0.29      0.33      0.31       251
weighted avg       0.78      0.88      0.83       251



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**3 слоя: 128, 64, 16**

Sigmoid

Adam

In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(32, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 16)
        self.fc4 = nn.Linear(16, 3)

    def forward(self, x):
        x = F.sigmoid(self.fc1(x))
        x = F.sigmoid(self.fc2(x))
        x = F.sigmoid(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)

In [ ]:
model = MLP()

# Определение функции потерь и оптимизатора
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Обучение модели
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in trainset:
        optimizer.zero_grad()
        outputs = model(inputs.float())
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(trainset)}")

Epoch 1/10, Loss: 0.3846906095385347
Epoch 2/10, Loss: 0.3594950431667102
Epoch 3/10, Loss: 0.35795022805395504
Epoch 4/10, Loss: 0.35690176874833696
Epoch 5/10, Loss: 0.35110183360492797
Epoch 6/10, Loss: 0.34441483839962944
Epoch 7/10, Loss: 0.3442702827141277
Epoch 8/10, Loss: 0.34159877698862123
Epoch 9/10, Loss: 0.3394606909099615
Epoch 10/10, Loss: 0.3325412553410792


In [ ]:
model.eval()  # Устанавливаем модель в режим оценки (выключаем режим обучения)

all_predictions = []
all_targets = []

with torch.no_grad():
    for inputs, labels in testset:
        outputs = model(inputs.float())
        _, predicted = torch.max(outputs, 1)
        all_predictions.extend(predicted.cpu().numpy())
        all_targets.extend(labels.cpu().numpy())

acc_exp1_3 = accuracy_score(all_targets, all_predictions)
f1_exp1_3 = f1_score(all_targets, all_predictions, average='weighted')
print(classification_report(all_targets, all_predictions))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       0.88      1.00      0.94       222
           2       0.00      0.00      0.00        21

    accuracy                           0.88       251
   macro avg       0.29      0.33      0.31       251
weighted avg       0.78      0.88      0.83       251



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**3 слоя: 128, 64, 16**

ReLu

SGD

In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(32, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 16)
        self.fc4 = nn.Linear(16, 3)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)

In [ ]:
model = MLP()

# Определение функции потерь и оптимизатора
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [ ]:
# Обучение модели
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in trainset:
        optimizer.zero_grad()
        outputs = model(inputs.float())
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(trainset)}")

Epoch 1/10, Loss: 0.42264024980603393
Epoch 2/10, Loss: 0.3918312231428839
Epoch 3/10, Loss: 0.371017128912366
Epoch 4/10, Loss: 0.3649707546364987
Epoch 5/10, Loss: 0.3579692181683043
Epoch 6/10, Loss: 0.35066959431861
Epoch 7/10, Loss: 0.34933077702167015
Epoch 8/10, Loss: 0.33715216647717927
Epoch 9/10, Loss: 0.3372858548882876
Epoch 10/10, Loss: 0.33303455282122707


In [ ]:
model.eval()  # Устанавливаем модель в режим оценки (выключаем режим обучения)

all_predictions = []
all_targets = []

with torch.no_grad():
    for inputs, labels in testset:
        outputs = model(inputs.float())
        _, predicted = torch.max(outputs, 1)
        all_predictions.extend(predicted.cpu().numpy())
        all_targets.extend(labels.cpu().numpy())

acc_exp1_4 = accuracy_score(all_targets, all_predictions)
f1_exp1_4 = f1_score(all_targets, all_predictions, average='weighted')
print(classification_report(all_targets, all_predictions))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       0.88      1.00      0.94       222
           2       0.00      0.00      0.00        21

    accuracy                           0.88       251
   macro avg       0.29      0.33      0.31       251
weighted avg       0.78      0.88      0.83       251



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**3 слоя: 128, 64, 16**

Tanh

SGD

In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(32, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 16)
        self.fc4 = nn.Linear(16, 3)

    def forward(self, x):
        x = F.tanh(self.fc1(x))
        x = F.tanh(self.fc2(x))
        x = F.tanh(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)

In [ ]:
model = MLP()

# Определение функции потерь и оптимизатора
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [ ]:
# Обучение модели
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in trainset:
        optimizer.zero_grad()
        outputs = model(inputs.float())
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(trainset)}")

Epoch 1/10, Loss: 0.8732606485409352
Epoch 2/10, Loss: 0.637617666561853
Epoch 3/10, Loss: 0.5074146795845359
Epoch 4/10, Loss: 0.4412640096495123
Epoch 5/10, Loss: 0.40311056206811147
Epoch 6/10, Loss: 0.3916271136335607
Epoch 7/10, Loss: 0.38019561853202294
Epoch 8/10, Loss: 0.3642940626225688
Epoch 9/10, Loss: 0.36141527824960074
Epoch 10/10, Loss: 0.357128454548008


In [ ]:
model.eval()  # Устанавливаем модель в режим оценки (выключаем режим обучения)

all_predictions = []
all_targets = []

with torch.no_grad():
    for inputs, labels in testset:
        outputs = model(inputs.float())
        _, predicted = torch.max(outputs, 1)
        all_predictions.extend(predicted.cpu().numpy())
        all_targets.extend(labels.cpu().numpy())

acc_exp1_5 = accuracy_score(all_targets, all_predictions)
f1_exp1_5 = f1_score(all_targets, all_predictions, average='weighted')
print(classification_report(all_targets, all_predictions))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       0.88      1.00      0.94       222
           2       0.00      0.00      0.00        21

    accuracy                           0.88       251
   macro avg       0.29      0.33      0.31       251
weighted avg       0.78      0.88      0.83       251



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**3 слоя: 128, 64, 16**

Sigmoid

SGD

In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(32, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 16)
        self.fc4 = nn.Linear(16, 3)

    def forward(self, x):
        x = F.sigmoid(self.fc1(x))
        x = F.sigmoid(self.fc2(x))
        x = F.sigmoid(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)

In [ ]:
model = MLP()

# Определение функции потерь и оптимизатора
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [ ]:
# Обучение модели
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in trainset:
        optimizer.zero_grad()
        outputs = model(inputs.float())
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(trainset)}")

Epoch 1/10, Loss: 0.71054196219706
Epoch 2/10, Loss: 0.4113361761357322
Epoch 3/10, Loss: 0.3692804583656645
Epoch 4/10, Loss: 0.36019667416913226
Epoch 5/10, Loss: 0.35758831513020023
Epoch 6/10, Loss: 0.3568932391434755
Epoch 7/10, Loss: 0.3565739623269331
Epoch 8/10, Loss: 0.35645949201559324
Epoch 9/10, Loss: 0.35634266110983764
Epoch 10/10, Loss: 0.3563741552379896


In [ ]:
model.eval()  # Устанавливаем модель в режим оценки (выключаем режим обучения)

all_predictions = []
all_targets = []

with torch.no_grad():
    for inputs, labels in testset:
        outputs = model(inputs.float())
        _, predicted = torch.max(outputs, 1)
        all_predictions.extend(predicted.cpu().numpy())
        all_targets.extend(labels.cpu().numpy())

acc_exp1_6 = accuracy_score(all_targets, all_predictions)
f1_exp1_6 = f1_score(all_targets, all_predictions, average='weighted')
print(classification_report(all_targets, all_predictions))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       0.88      1.00      0.94       222
           2       0.00      0.00      0.00        21

    accuracy                           0.88       251
   macro avg       0.29      0.33      0.31       251
weighted avg       0.78      0.88      0.83       251



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**3 слоя: 128, 64, 16**

ReLu

RMSprop

In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(32, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 16)
        self.fc4 = nn.Linear(16, 3)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)

In [ ]:
model = MLP()

# Определение функции потерь и оптимизатора
criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(model.parameters(), lr=0.001)

In [ ]:
# Обучение модели
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in trainset:
        optimizer.zero_grad()
        outputs = model(inputs.float())
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(trainset)}")

Epoch 1/10, Loss: 0.6478850355388992
Epoch 2/10, Loss: 0.40511486120167833
Epoch 3/10, Loss: 0.40815820563968663
Epoch 4/10, Loss: 0.37415889180368667
Epoch 5/10, Loss: 0.3790622824376324
Epoch 6/10, Loss: 0.3668132081066086
Epoch 7/10, Loss: 0.3592604411273553
Epoch 8/10, Loss: 0.39766922444543
Epoch 9/10, Loss: 0.3712729149943255
Epoch 10/10, Loss: 0.34424631736109346


In [ ]:
model.eval()  # Устанавливаем модель в режим оценки (выключаем режим обучения)

all_predictions = []
all_targets = []

with torch.no_grad():
    for inputs, labels in testset:
        outputs = model(inputs.float())
        _, predicted = torch.max(outputs, 1)
        all_predictions.extend(predicted.cpu().numpy())
        all_targets.extend(labels.cpu().numpy())

acc_exp1_7 = accuracy_score(all_targets, all_predictions)
f1_exp1_7 = f1_score(all_targets, all_predictions, average='weighted')
print(classification_report(all_targets, all_predictions))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       0.88      1.00      0.94       222
           2       0.00      0.00      0.00        21

    accuracy                           0.88       251
   macro avg       0.29      0.33      0.31       251
weighted avg       0.78      0.88      0.83       251



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**3 слоя: 128, 64, 16**

Tanh

RMSprop

In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(32, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 16)
        self.fc4 = nn.Linear(16, 3)

    def forward(self, x):
        x = F.tanh(self.fc1(x))
        x = F.tanh(self.fc2(x))
        x = F.tanh(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)

In [ ]:
model = MLP()

# Определение функции потерь и оптимизатора
criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(model.parameters(), lr=0.001)

In [ ]:
# Обучение модели
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in trainset:
        optimizer.zero_grad()
        outputs = model(inputs.float())
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(trainset)}")

Epoch 1/10, Loss: 0.4706934007590774
Epoch 2/10, Loss: 0.36809674733498904
Epoch 3/10, Loss: 0.3635969248122662
Epoch 4/10, Loss: 0.3521125548763606
Epoch 5/10, Loss: 0.350812768292059
Epoch 6/10, Loss: 0.33911868333049744
Epoch 7/10, Loss: 0.3392009035034
Epoch 8/10, Loss: 0.3354989162056769
Epoch 9/10, Loss: 0.3325453735249018
Epoch 10/10, Loss: 0.3286542715712436


In [ ]:
model.eval()  # Устанавливаем модель в режим оценки (выключаем режим обучения)

all_predictions = []
all_targets = []

with torch.no_grad():
    for inputs, labels in testset:
        outputs = model(inputs.float())
        _, predicted = torch.max(outputs, 1)
        all_predictions.extend(predicted.cpu().numpy())
        all_targets.extend(labels.cpu().numpy())

acc_exp1_8 = accuracy_score(all_targets, all_predictions)
f1_exp1_8 = f1_score(all_targets, all_predictions, average='weighted')
print(classification_report(all_targets, all_predictions))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       0.88      1.00      0.94       222
           2       0.00      0.00      0.00        21

    accuracy                           0.88       251
   macro avg       0.29      0.33      0.31       251
weighted avg       0.78      0.88      0.83       251



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**3 слоя: 128, 64, 16**

Sigmoid

RMSprop

In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(32, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 16)
        self.fc4 = nn.Linear(16, 3)

    def forward(self, x):
        x = F.sigmoid(self.fc1(x))
        x = F.sigmoid(self.fc2(x))
        x = F.sigmoid(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)

In [ ]:
model = MLP()

# Определение функции потерь и оптимизатора
criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(model.parameters(), lr=0.001)

In [ ]:
# Обучение модели
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in trainset:
        optimizer.zero_grad()
        outputs = model(inputs.float())
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(trainset)}")

Epoch 1/10, Loss: 0.36876581938327074
Epoch 2/10, Loss: 0.3600715284473798
Epoch 3/10, Loss: 0.3580855626172001
Epoch 4/10, Loss: 0.3558118236195264
Epoch 5/10, Loss: 0.34938524361846784
Epoch 6/10, Loss: 0.3542173554350847
Epoch 7/10, Loss: 0.3476953371963104
Epoch 8/10, Loss: 0.3414949778512146
Epoch 9/10, Loss: 0.34295585999460204
Epoch 10/10, Loss: 0.34010883729336194


In [ ]:
model.eval()  # Устанавливаем модель в режим оценки (выключаем режим обучения)

all_predictions = []
all_targets = []

with torch.no_grad():
    for inputs, labels in testset:
        outputs = model(inputs.float())
        _, predicted = torch.max(outputs, 1)
        all_predictions.extend(predicted.cpu().numpy())
        all_targets.extend(labels.cpu().numpy())

acc_exp1_9 = accuracy_score(all_targets, all_predictions)
f1_exp1_9 = f1_score(all_targets, all_predictions, average='weighted')
print(classification_report(all_targets, all_predictions))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       0.88      1.00      0.94       222
           2       0.00      0.00      0.00        21

    accuracy                           0.88       251
   macro avg       0.29      0.33      0.31       251
weighted avg       0.78      0.88      0.83       251



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# **Результаты**

In [ ]:
layers_ = [
    '128, 64, 16',
    '128, 64, 16',
    '128, 64, 16',
    '128, 64, 16',
    '128, 64, 16',
    '128, 64, 16',
    '128, 64, 16',
    '128, 64, 16',
    '128, 64, 16'
]

activation_function_ = [
    'Relu',
    'Tanh',
    'Sigmoid',
    'Relu',
    'Tanh',
    'Sigmoid',
    'Relu',
    'Tanh',
    'Sigmoid'
]

optimizer_ = [
    'Adam',
    'Adam',
    'Adam',
    'SGD',
    'SGD',
    'SGD',
    'RMSprop',
    'RMSprop',
    'RMSprop'
]

accuracy_ = [
    acc_exp1_1,
    acc_exp1_2,
    acc_exp1_3,
    acc_exp1_4,
    acc_exp1_5,
    acc_exp1_6,
    acc_exp1_7,
    acc_exp1_8,
    acc_exp1_9
]

f1_score_ = [
    f1_exp1_1,
    f1_exp1_2,
    f1_exp1_3,
    f1_exp1_4,
    f1_exp1_5,
    f1_exp1_6,
    f1_exp1_7,
    f1_exp1_8,
    f1_exp1_9
]

In [ ]:
results_text_df = pd.DataFrame({
    'Слои': layers_,
    'Функция активации': activation_function_,
    'Оптимизатор': optimizer_,
    'Accuracy': accuracy_,
    'F1 score': f1_score_
})

In [ ]:
results_audio_df = pd.DataFrame({
    'Слои': layers_,
    'Функция активации': activation_function_,
    'Оптимизатор': optimizer_,
    'Accuracy': accuracy_,
    'F1 score': f1_score_
})

In [ ]:
results_text_df

,Слои,Функция активации,Оптимизатор,Accuracy,F1 score
0,"128, 64, 16",Relu,Adam,0.868526,0.852090
1,"128, 64, 16",Tanh,Adam,0.868526,0.855986
2,"128, 64, 16",Sigmoid,Adam,0.884462,0.830235
3,"128, 64, 16",Relu,SGD,0.884462,0.830235
4,"128, 64, 16",Tanh,SGD,0.884462,0.830235
5,"128, 64, 16",Sigmoid,SGD,0.884462,0.830235
6,"128, 64, 16",Relu,RMSprop,0.872510,0.846466
7,"128, 64, 16",Tanh,RMSprop,0.872510,0.859032
8,"128, 64, 16",Sigmoid,RMSprop,0.884462,0.830235


7 - лучший результат по F1 score

In [ ]:
results_audio_df

,Слои,Функция активации,Оптимизатор,Accuracy,F1 score
0,"128, 64, 16",Relu,Adam,0.884462,0.830235
1,"128, 64, 16",Tanh,Adam,0.880478,0.828246
2,"128, 64, 16",Sigmoid,Adam,0.884462,0.830235
3,"128, 64, 16",Relu,SGD,0.884462,0.830235
4,"128, 64, 16",Tanh,SGD,0.884462,0.830235
5,"128, 64, 16",Sigmoid,SGD,0.884462,0.830235
6,"128, 64, 16",Relu,RMSprop,0.884462,0.830235
7,"128, 64, 16",Tanh,RMSprop,0.880478,0.828246
8,"128, 64, 16",Sigmoid,RMSprop,0.884462,0.830235
